In [1]:
import json
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.utils import pad_sequences
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.layers import Masking
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, GlobalMaxPooling1D, Dense, Dropout, Masking, Input, Bidirectional
from keras.regularizers import l2
import tensorflow as tf
import sys

In [26]:
# Load model
import lightgbm as lgb
from lightgbm import LGBMClassifier
model_domain2 = lgb.Booster(model_file='domain2_model.txt')


In [3]:
from tensorflow.keras.models import load_model

model_domain = load_model("model_domain")

model_domain1 = load_model("model_domain1")

In [9]:
test_texts = []
test_texts_tfidf = []
zeros_in_text = []
import pickle
import json

def dummy_fun(doc):
    return doc

# Open file for reading
with open('test_set.json', 'r') as f:
    for line in f:
        # Parse the JSON line into a Python dictionary
        obj = json.loads(line)        
        text_without_zeros = []
        count_zeros = 0
        for id in obj['text']:
            if id != 0 :
                text_without_zeros.append(id)
            else:
                count_zeros += 1
        test_texts_tfidf.append(obj['text'])
        test_texts.append(text_without_zeros)
        zeros_in_text.append(count_zeros)

padded_texts = pad_sequences(test_texts, padding='pre', value=0)

tfidf = TfidfVectorizer(
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  
# Load it back
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

test_texts_tfidf = tfidf.transform(test_texts_tfidf)
zeros_in_text = np.array(zeros_in_text).reshape(-1, 1)

In [14]:
# Make predictions
predictions = model_domain.predict([padded_texts, zeros_in_text])

# Interpret predictions
domain_predictions = np.argmax(predictions, axis=1)

32/32 [==============================] - 1s 17ms/step


In [23]:
model1_predictions = model_domain1.predict([padded_texts, zeros_in_text])
# Interpret predictions
model1_predictions = np.argmax(predictions, axis=1)

32/32 [==============================] - 20s 553ms/step


In [27]:
# Make predictions
model2_predictions = model_domain2.predict(test_texts_tfidf)
# Convert to binary output
model2_predictions = [1 if pred > 0.5 else 0 for pred in model2_predictions]

In [29]:
final_predictions =[]
count_domain_1 = 0
count_domain_2 = 0
for i in range(len(domain_predictions)):
    domain = domain_predictions[i]
    if domain == 0:
        final_predictions.append(model1_predictions[i])
        count_domain_1+=1
    else:
        final_predictions.append(model2_predictions[i])
        count_domain_2+=1


In [37]:
import pandas as pd
output_df = pd.DataFrame({"id":list(range(len(final_predictions))), "class": final_predictions})
output_df.to_csv("output.csv", index=False)

In [31]:
count_domain_2

503